## NMPC vs Feedforward Tracking Comparison
Compare controller tracking performance using rosbag data.
- **Mode A**: Single bag — setpoint vs actual odometry (tracking error for one run)
- **Mode B**: Two bags — feedforward vs NMPC side-by-side comparison

In [ ]:
!pip install -q rosbag2-py rclpy scipy matplotlib numpy==1.25.0 pandas

### Bag Reading Utilities

In [ ]:
import rosbag2_py
import rclpy
from rclpy.serialization import deserialize_message
from rosidl_runtime_py.utilities import get_message
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R
from scipy.interpolate import interp1d
import pandas as pd

try:
    rclpy.init()
except:
    pass


def read_topic(bag_path, topic_name):
    """Read all messages from a single topic, return (timestamps_sec, messages)."""
    reader = rosbag2_py.SequentialReader()
    reader.open(
        rosbag2_py.StorageOptions(uri=bag_path, storage_id='sqlite3'),
        rosbag2_py.ConverterOptions(input_serialization_format='cdr', output_serialization_format='cdr'),
    )
    type_map = {t.name: t.type for t in reader.get_all_topics_and_types()}
    if topic_name not in type_map:
        print(f"Topic {topic_name} not found in bag. Available: {list(type_map.keys())}")
        return None, None
    msg_type = get_message(type_map[topic_name])
    reader.set_filter(rosbag2_py.StorageFilter(topics=[topic_name]))
    timestamps, messages = [], []
    while reader.has_next():
        topic, data, t_ns = reader.read_next()
        timestamps.append(t_ns / 1e9)
        messages.append(deserialize_message(data, msg_type))
    return np.array(timestamps), messages


def extract_odom(messages, timestamps):
    """Extract position and velocity arrays from Odometry messages."""
    pos, vel = [], []
    for msg in messages:
        p = msg.pose.pose.position
        v = msg.twist.twist.linear
        pos.append([p.x, p.y, p.z])
        vel.append([v.x, v.y, v.z])
    return np.array(pos), np.array(vel)


def extract_setpoint(messages, timestamps):
    """Extract position and velocity arrays from PositionTarget messages."""
    pos, vel = [], []
    for msg in messages:
        pos.append([msg.position.x, msg.position.y, msg.position.z])
        vel.append([msg.velocity.x, msg.velocity.y, msg.velocity.z])
    return np.array(pos), np.array(vel)


def interpolate_to_common_time(ts_a, data_a, ts_b, data_b):
    """Interpolate two time series onto the overlapping time range at the slower rate."""
    t_start = max(ts_a[0], ts_b[0])
    t_end = min(ts_a[-1], ts_b[-1])
    mask_a = (ts_a >= t_start) & (ts_a <= t_end)
    mask_b = (ts_b >= t_start) & (ts_b <= t_end)
    t_common = ts_a[mask_a]  # use odom timestamps as reference
    if len(t_common) == 0:
        print("No overlapping time range found.")
        return None, None, None
    interp_funcs = [interp1d(ts_b, data_b[:, i], kind='linear', fill_value='extrapolate') for i in range(data_b.shape[1])]
    data_b_interp = np.column_stack([f(t_common) for f in interp_funcs])
    data_a_matched = data_a[mask_a]
    return t_common - t_common[0], data_a_matched, data_b_interp


def compute_metrics(error_3d):
    """Compute RMSE, MAE, and max error from a 1D error-norm array."""
    return {
        'RMSE (m)': np.sqrt(np.mean(error_3d ** 2)),
        'MAE (m)': np.mean(np.abs(error_3d)),
        'Max (m)': np.max(np.abs(error_3d)),
        'Std (m)': np.std(error_3d),
    }


print("Utilities loaded.")

### Mode A — Single Bag Tracking Error
Compare controller setpoints vs actual odometry for a single run.

In [ ]:
# ===== CONFIGURE =====
bag_path = "../bags/test1_sim"  # <-- change to your bag path
setpoint_topic = "mavros/setpoint_raw/local"
odom_topic = "mavros/odometry/in"  # or mavros/odometry/out
max_time = None  # set e.g. 60 to limit to first 60s, or None for full bag
# =====================

ts_sp, msgs_sp = read_topic(bag_path, setpoint_topic)
ts_od, msgs_od = read_topic(bag_path, odom_topic)

if ts_sp is not None and ts_od is not None:
    sp_pos, sp_vel = extract_setpoint(msgs_sp, ts_sp)
    od_pos, od_vel = extract_odom(msgs_od, ts_od)

    t_common, od_pos_c, sp_pos_c = interpolate_to_common_time(ts_od, od_pos, ts_sp, sp_pos)
    _, od_vel_c, sp_vel_c = interpolate_to_common_time(ts_od, od_vel, ts_sp, sp_vel)

    if t_common is not None:
        if max_time:
            mask = t_common <= max_time
            t_common, od_pos_c, sp_pos_c = t_common[mask], od_pos_c[mask], sp_pos_c[mask]
            od_vel_c, sp_vel_c = od_vel_c[mask], sp_vel_c[mask]

        pos_err = sp_pos_c - od_pos_c
        pos_err_norm = np.linalg.norm(pos_err, axis=1)
        vel_err = sp_vel_c - od_vel_c
        vel_err_norm = np.linalg.norm(vel_err, axis=1)

        # --- Position tracking error ---
        fig, axes = plt.subplots(2, 2, figsize=(14, 9))
        labels = ['X', 'Y', 'Z']
        for i in range(3):
            axes[0, 0].plot(t_common, pos_err[:, i], label=f'{labels[i]}', linewidth=1)
        axes[0, 0].set_title('Position Tracking Error (per axis)')
        axes[0, 0].set_ylabel('Error (m)'); axes[0, 0].legend(); axes[0, 0].grid(True, alpha=0.3)

        axes[0, 1].plot(t_common, pos_err_norm, 'k-', linewidth=1)
        axes[0, 1].set_title('3D Position Error Norm')
        axes[0, 1].set_ylabel('Error (m)'); axes[0, 1].grid(True, alpha=0.3)

        for i in range(3):
            axes[1, 0].plot(t_common, vel_err[:, i], label=f'V{labels[i].lower()}', linewidth=1)
        axes[1, 0].set_title('Velocity Tracking Error (per axis)')
        axes[1, 0].set_xlabel('Time (s)'); axes[1, 0].set_ylabel('Error (m/s)')
        axes[1, 0].legend(); axes[1, 0].grid(True, alpha=0.3)

        axes[1, 1].hist(pos_err_norm, bins=50, edgecolor='black', alpha=0.7)
        axes[1, 1].set_title('Position Error Distribution')
        axes[1, 1].set_xlabel('Error (m)'); axes[1, 1].set_ylabel('Count')
        axes[1, 1].grid(True, alpha=0.3)

        plt.suptitle('Single Bag Tracking Error Analysis', fontsize=14)
        plt.tight_layout(); plt.show()

        # --- Position overlay ---
        fig, axes = plt.subplots(1, 3, figsize=(16, 4))
        for i, lbl in enumerate(labels):
            axes[i].plot(t_common, sp_pos_c[:, i], 'r-', label='Setpoint', linewidth=1.5)
            axes[i].plot(t_common, od_pos_c[:, i], 'b-', label='Odom', linewidth=1, alpha=0.8)
            axes[i].set_title(f'{lbl} Position'); axes[i].set_xlabel('Time (s)')
            axes[i].set_ylabel('m'); axes[i].legend(); axes[i].grid(True, alpha=0.3)
        plt.tight_layout(); plt.show()

        # --- Metrics ---
        metrics = compute_metrics(pos_err_norm)
        vel_metrics = compute_metrics(vel_err_norm)
        print("\n=== Position Tracking Metrics ===")
        for k, v in metrics.items(): print(f"  {k}: {v:.4f}")
        print("\n=== Velocity Tracking Metrics ===")
        for k, v in vel_metrics.items(): print(f"  {k}: {v:.4f}")

### Mode B — Two Bag Comparison (Feedforward vs NMPC)
Load two bags recorded with the same waypoints but different `controller_mode`, and compare tracking error.

In [ ]:
# ===== CONFIGURE =====
bag_ff = "../bags/feedforward_run"  # <-- feedforward bag
bag_nmpc = "../bags/nmpc_run"       # <-- nmpc bag
setpoint_topic = "mavros/setpoint_raw/local"
odom_topic = "mavros/odometry/in"
max_time = None
# =====================

def load_tracking_data(bag_path, sp_topic, od_topic, max_t=None):
    ts_sp, msgs_sp = read_topic(bag_path, sp_topic)
    ts_od, msgs_od = read_topic(bag_path, od_topic)
    if ts_sp is None or ts_od is None:
        return None
    sp_pos, sp_vel = extract_setpoint(msgs_sp, ts_sp)
    od_pos, od_vel = extract_odom(msgs_od, ts_od)
    t_c, od_p, sp_p = interpolate_to_common_time(ts_od, od_pos, ts_sp, sp_pos)
    _, od_v, sp_v = interpolate_to_common_time(ts_od, od_vel, ts_sp, sp_vel)
    if t_c is None:
        return None
    if max_t:
        mask = t_c <= max_t
        t_c, od_p, sp_p, od_v, sp_v = t_c[mask], od_p[mask], sp_p[mask], od_v[mask], sp_v[mask]
    pos_err = np.linalg.norm(sp_p - od_p, axis=1)
    vel_err = np.linalg.norm(sp_v - od_v, axis=1)
    return {'t': t_c, 'odom_pos': od_p, 'sp_pos': sp_p, 'odom_vel': od_v, 'sp_vel': sp_v,
            'pos_err': pos_err, 'vel_err': vel_err, 'pos_err_axis': sp_p - od_p}


ff_data = load_tracking_data(bag_ff, setpoint_topic, odom_topic, max_time)
nmpc_data = load_tracking_data(bag_nmpc, setpoint_topic, odom_topic, max_time)

if ff_data and nmpc_data:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # 3D position error comparison
    axes[0, 0].plot(ff_data['t'], ff_data['pos_err'], 'b-', label='Feedforward', linewidth=1, alpha=0.8)
    axes[0, 0].plot(nmpc_data['t'], nmpc_data['pos_err'], 'r-', label='NMPC', linewidth=1, alpha=0.8)
    axes[0, 0].set_title('3D Position Error Norm'); axes[0, 0].set_ylabel('Error (m)')
    axes[0, 0].legend(); axes[0, 0].grid(True, alpha=0.3)

    # Per-axis error comparison (X)
    for i, lbl in enumerate(['X', 'Y', 'Z']):
        axes[0, 1].plot(ff_data['t'], np.abs(ff_data['pos_err_axis'][:, i]),
                        linestyle='-', alpha=0.6, label=f'FF {lbl}')
    for i, lbl in enumerate(['X', 'Y', 'Z']):
        axes[0, 1].plot(nmpc_data['t'], np.abs(nmpc_data['pos_err_axis'][:, i]),
                        linestyle='--', alpha=0.6, label=f'NMPC {lbl}')
    axes[0, 1].set_title('Per-Axis Position Error (abs)'); axes[0, 1].set_ylabel('Error (m)')
    axes[0, 1].legend(fontsize=8, ncol=2); axes[0, 1].grid(True, alpha=0.3)

    # Velocity error
    axes[1, 0].plot(ff_data['t'], ff_data['vel_err'], 'b-', label='Feedforward', linewidth=1, alpha=0.8)
    axes[1, 0].plot(nmpc_data['t'], nmpc_data['vel_err'], 'r-', label='NMPC', linewidth=1, alpha=0.8)
    axes[1, 0].set_title('3D Velocity Error Norm'); axes[1, 0].set_xlabel('Time (s)')
    axes[1, 0].set_ylabel('Error (m/s)'); axes[1, 0].legend(); axes[1, 0].grid(True, alpha=0.3)

    # Error distribution
    axes[1, 1].hist(ff_data['pos_err'], bins=50, alpha=0.5, label='Feedforward', edgecolor='blue')
    axes[1, 1].hist(nmpc_data['pos_err'], bins=50, alpha=0.5, label='NMPC', edgecolor='red')
    axes[1, 1].set_title('Position Error Distribution'); axes[1, 1].set_xlabel('Error (m)')
    axes[1, 1].set_ylabel('Count'); axes[1, 1].legend(); axes[1, 1].grid(True, alpha=0.3)

    plt.suptitle('Feedforward vs NMPC Tracking Comparison', fontsize=14)
    plt.tight_layout(); plt.show()

    # 3D trajectory overlay
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot(ff_data['odom_pos'][:, 0], ff_data['odom_pos'][:, 1], ff_data['odom_pos'][:, 2],
            'b-', label='FF Actual', linewidth=1.5)
    ax.plot(ff_data['sp_pos'][:, 0], ff_data['sp_pos'][:, 1], ff_data['sp_pos'][:, 2],
            'b--', label='FF Setpoint', linewidth=1, alpha=0.5)
    ax.plot(nmpc_data['odom_pos'][:, 0], nmpc_data['odom_pos'][:, 1], nmpc_data['odom_pos'][:, 2],
            'r-', label='NMPC Actual', linewidth=1.5)
    ax.plot(nmpc_data['sp_pos'][:, 0], nmpc_data['sp_pos'][:, 1], nmpc_data['sp_pos'][:, 2],
            'r--', label='NMPC Setpoint', linewidth=1, alpha=0.5)
    ax.set_xlabel('X (m)'); ax.set_ylabel('Y (m)'); ax.set_zlabel('Z (m)')
    ax.set_title('3D Trajectory Overlay'); ax.legend(); ax.grid(True)
    plt.tight_layout(); plt.show()
else:
    print("Failed to load one or both bags. Check paths above.")

### Summary Metrics Table

In [ ]:
if ff_data and nmpc_data:
    rows = []
    for name, data in [('Feedforward', ff_data), ('NMPC', nmpc_data)]:
        pm = compute_metrics(data['pos_err'])
        vm = compute_metrics(data['vel_err'])
        rows.append({
            'Controller': name,
            'Pos RMSE (m)': f"{pm['RMSE (m)']:.4f}",
            'Pos MAE (m)': f"{pm['MAE (m)']:.4f}",
            'Pos Max (m)': f"{pm['Max (m)']:.4f}",
            'Pos Std (m)': f"{pm['Std (m)']:.4f}",
            'Vel RMSE (m/s)': f"{vm['RMSE (m)']:.4f}",
            'Vel MAE (m/s)': f"{vm['MAE (m)']:.4f}",
            'Vel Max (m/s)': f"{vm['Max (m)']:.4f}",
        })
    df_summary = pd.DataFrame(rows)
    display(df_summary)
else:
    print("Run Mode B first to generate comparison data.")